In [25]:
import mido

def preview_midi_tracks(midi_file: str):
    """
    Previews the tracks in a MIDI file, including their names and events.
    """
    mid = mido.MidiFile(midi_file)

    for i, track in enumerate(mid.tracks):
        track_name = f'Track {i + 1}'
        if len(track) < 10:
            continue
        for msg in track:
            if msg.type == 'track_name':
                track_name = msg.name
                break  # Track name found, no need to look further
        print(f'{i}) {track_name}: [{len(track)} messages]')
        
preview_midi_tracks("../Melodies/fur_Elise_WoO59.mid")

1) up:: [1039 messages]
2) down:: [780 messages]


In [26]:
def midi_to_note_name(midi_note: int) -> str:
    """
    Converts a MIDI note number to its corresponding note name (e.g., 60 to 'C5').

    Args:
        midi_note: The MIDI note number (0-127).

    Returns:
        The note name as a string.
        Returns an empty string if the input is invalid.
    """
    if 0 <= midi_note <= 127:
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        octave = (midi_note // 12) - 1
        note_index = midi_note % 12
        return f"{notes[note_index]}{octave}"
    else:
        return ""
        #  out of range exception?


midi_to_note_name(60)

'C4'

In [46]:
def midi_to_notes_and_timing(midi_file: str, track_index: int = 1, monophonic: bool = True):
    """
    Extracts monophonic notes and their timings from a specific track of a MIDI file.

    Args:
        midi_file: Path to the MIDI file.
        track_index: The index of the track to extract information from (default is 0).
        monophonic: If True, only the last played note will be considered when multiple notes overlap.

    Returns:
        A list of tuples, where each tuple contains (note_name, duration_seconds).
        Returns an empty list if the track is invalid or contains no note events.
    """
    try:
        mid = mido.MidiFile(midi_file)
        if track_index >= len(mid.tracks):
            print(f"Error: Track index {track_index} is out of bounds.")
            return []
        track = mid.tracks[track_index]

        notes_on = {}  # Keep track of currently playing notes (note_number: start_time)
        notes_and_timing = []
        current_time = 0
        previous_end_time = 0  # Keep track of the end time of the previous note
        ticks_per_beat = mid.ticks_per_beat
        tempo = 500000  # Default tempo (microseconds per beat)

        for msg in track:
            current_time += msg.time

            if msg.type == 'set_tempo':
                tempo = msg.tempo

            if msg.type == 'note_on':
                if msg.velocity > 0:  # Note on event
                    # Check for a pause before this note
                    if monophonic and current_time > previous_end_time and previous_end_time != 0:
                        pause_duration = mido.tick2second(current_time - previous_end_time, ticks_per_beat, tempo)
                        notes_and_timing.append(("P", pause_duration))

                    notes_on[msg.note] = current_time
                else:  # Note off event with velocity 0 (alternative to 'note_off' message)
                    if msg.note in notes_on:
                        start_time = notes_on.pop(msg.note)
                        duration_ticks = current_time - start_time
                        duration_seconds = mido.tick2second(duration_ticks, ticks_per_beat, tempo)
                        note_name = midi_to_note_name(msg.note)
                        notes_and_timing.append((note_name, duration_seconds))
                        previous_end_time = current_time # Update the end time

            elif msg.type == 'note_off':
                if msg.note in notes_on:
                    start_time = notes_on.pop(msg.note)
                    duration_ticks = current_time - start_time
                    duration_seconds = mido.tick2second(duration_ticks, ticks_per_beat, tempo)
                    note_name = midi_to_note_name(msg.note)
                    notes_and_timing.append((note_name, duration_seconds))
                    previous_end_time = current_time  # Update the end time

            # Handle monophonic constraint: if a new note starts while another is playing,
            # consider the previous one ended immediately.
            if monophonic and msg.type == 'note_on' and msg.velocity > 0:
                if notes_on:
                    previous_note = list(notes_on.keys())[0]  # Get the currently playing note
                    start_time = notes_on.pop(previous_note)
                    duration_ticks = current_time - start_time
                    duration_seconds = mido.tick2second(duration_ticks, ticks_per_beat, tempo)
                    note_name = midi_to_note_name(previous_note)
                    notes_and_timing.append((note_name, duration_seconds))
                    previous_end_time = current_time # consider previous note ended.
                notes_on[msg.note] = current_time  # Start the new note

        # Handle any remaining notes that were on at the end of the track
        for note, start_time in notes_on.items():
            duration_ticks = current_time - start_time
            duration_seconds = mido.tick2second(duration_ticks, ticks_per_beat, tempo)
            note_name = midi_to_note_name(note)
            notes_and_timing.append((note_name, duration_seconds))
            previous_end_time = current_time

        return notes_and_timing

    except FileNotFoundError:
        print(f"Error: MIDI file '{midi_file}' not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []



midi_to_notes_and_timing("../Melodies/MoneyForNothing.mid",14)[:20]

[('D4', 0.0),
 ('D4', 0.109375),
 ('P', 0.390625),
 ('D4', 0.0),
 ('D4', 0.4895833333333333),
 ('P', 0.010416666666666666),
 ('D4', 0.0),
 ('D4', 0.234375),
 ('P', 0.015625),
 ('C4', 0.0),
 ('C4', 0.24479166666666666),
 ('P', 0.005208333333333333),
 ('D4', 0.0),
 ('D4', 0.23958333333333331),
 ('P', 0.010416666666666666),
 ('F4', 0.0),
 ('F4', 0.234375),
 ('P', 0.015625),
 ('F4', 0.0),
 ('F4', 0.484375)]

In [49]:
def display_notes_and_pauses(notes_timing: list):
    """
    Displays the notes and pauses in the desired format (e.g., "C4:0.4", "P:0.1").

    Args:
        notes_timing: A list of tuples (note_name, duration_seconds).
    """
    if not notes_timing:
        print("No notes and timings to display.")
        return

    formatted_output = []
    for note, duration in notes_timing:
        if note == "P":
            print(f"P:{duration:.2f}")
        elif duration:
            print(f"{note}:{duration:.2f}")
        else:
            pass
    print("\n".join(formatted_output))



if __name__ == '__main__':
    midi_file = "../Melodies/Stairway to heaven.mid"
    melody_track_index = 16

    monophonic_notes = midi_to_notes_and_timing(midi_file, melody_track_index, monophonic=True)

    if monophonic_notes:
#         print(f"\nNotes and pauses from Track {melody_track_index}:")
        display_notes_and_pauses(monophonic_notes)


P:0.24
A3:0.24
C4:0.26
P:0.24
E4:0.24
A4:0.26
P:0.24
B4:0.24
P:0.24
E4:0.24
C4:0.26
B4:0.25
C5:0.25
E4:0.25
P:0.23
C4:0.23
C5:0.13
P:0.14
F#4:0.25
D4:0.25
A3:0.25
F#4:0.25
P:0.24
E4:0.24
P:0.25
C4:0.25
A3:0.26
E4:0.41
P:0.08
P:0.24
E4:0.24
P:0.25
C4:0.25
A3:0.13
P:0.13
G3:0.28
A3:0.09
P:0.12
A3:0.76
P:0.24
A2:0.24
F3:0.26
P:0.24
E3:0.24
P:0.26
A2:0.26
P:0.25
C4:0.25
P:0.24
E4:0.24
A4:0.25
P:0.26
B4:0.26
E4:0.25
C4:0.25
P:0.25
B4:0.25
C5:0.25
E4:0.25
C4:0.25
C5:0.14
P:0.11
F#4:0.25
D4:0.25
A3:0.25
F#4:0.24
P:0.01
P:0.24
E4:0.24
P:0.25
C4:0.25
A3:0.26
P:0.49
C4:0.49
P:0.24
E4:0.24
P:0.25
C4:0.25
A3:0.11
P:0.15
G3:0.25
A3:0.06
P:0.19
A3:0.81

